### 1. Set up graph

In [10]:
import networkx as nx
import matplotlib.pyplot as plt
from pyvis.network import Network
import pandas as pd
import numpy as np

In [11]:
nodes=pd.read_csv("data/" + "nodes.csv")
edges=pd.read_csv("data/" + "edges.csv")

In [12]:
G=nx.DiGraph()

for idx, row in nodes.iterrows():
    G.add_node(idx, text=row["text"], ntype=row["ntype"])
    
for idx, row in edges.iterrows():
    G.add_edge(u_of_edge=row["origin"], v_of_edge=row["destination"], direction=row["direction"], title=row["title"])

### 2. Preprocessing for graph

#### 2.1 Define Labels

In [13]:
def splitLabels(s):
    #Splits labels to format for PyVis graph
    def findOccurrences(s, ch):
        return [i for i, letter in enumerate(s) if letter == ch]
    
    def newLinepos(labelPos, targetLength=20):
        l=[i % 20 for i in labelPos]
        return np.greater_equal(l[:-1], l[1:])
       
    pos=findOccurrences(s, " ")
    if bool(pos):
        newpos=newLinepos(pos)
        newpos=np.append([False], newpos)
        newpos=np.array(pos)[newpos]
        return "".join(["\n" if idx in newpos else i for idx, i in enumerate(s)])
    else:
        return s
print(f"Example: \n\nBefore {G.nodes[10]['text']} \n\nAfter {splitLabels(G.nodes[10]['text'])}")

Example: 

Before National infrastructure project funding 

After National infrastructure
project funding


In [14]:
mapping={node:splitLabels(prop["text"]) for node, prop in G.nodes(data=True)}
G = nx.relabel_nodes(G, mapping)

#Verify the graph has loaded successfully
print("The house prices node looks like %s" % (G.nodes["House prices"]))

The house prices node looks like {'text': 'House prices', 'ntype': 'M'}


#### 2.2: Assign node and edge properties

In [15]:
#Assign nodes colours
mapping={node:"rgb(255,128,0)"   if prop["ntype"]=="F" else 
              "rgb(102,178,255)" if prop["ntype"]=="M" else 
              "rgb(160,160,160)" 
              for node, prop in G.nodes(data=True)}
nx.set_node_attributes(G,mapping,'color')

#Assign edges colours
mapping={(start, end):"rgb(255,0,0)" if prop["direction"]=="down" else 
                      "rgb(0,0,255)" if prop["direction"]=="up" else
                      "rgb(160,160,160" 
                      for start, end, prop in G.edges(data=True)}
nx.set_edge_attributes(G,mapping,'color')

#Set node sizes
nx.set_node_attributes(G,25,'size')

#### 2.3: Configure PyVis graph

In [16]:
economy = Network(height='750px', width='75%', bgcolor='#222222', font_color='white', directed=True)
economy.toggle_physics(False)

#Take Networkx graph and translate it to a PyVis graph format
economy.from_nx(G)

# Generate network with specific layout settings
economy.repulsion(      node_distance=200,
                        central_gravity=0.33,
                        spring_length=110,
                        spring_strength=0.10,
                        damping=0.95
                  )
economy.save_graph("html_files/" + "economynetworkgraph.html")